### Example of surface traction integration for FEM (Bathe book, p. 361)

In [58]:
using SymPy

In [59]:
# Analytical integration using SymPy
r, p1, p2 = symbols("r, p1, p2")
N =  [r*(1+r) 0; 0 r*(1+r); -r*(1-r) 0; 0 -r*(1-r); 2*(1-r^2) 0; 0 2*(1-r^2)]
t =  [0; (1+ r)*p1+(1-r)*p2] 
int_Nt = integrate.(1/4*N*t, r)
int_Nt.subs(r,1).subs((p1,p2).=> (1,1)) .- int_Nt.subs(r,-1).subs((p1,p2).=> (1,1))

6×1 Matrix{Sym}:
                 0
 0.333333333333333
                 0
 0.333333333333333
                 0
  1.33333333333333

In [71]:
# Define integration points coordinates and weights
nip =  3
ipx = zeros(nip,1)
ipw = zeros(nip,1)
# Location
ipx[1] = -.775 #+.5
ipx[2] = 0.0# +.5
ipx[3] = .775 #+.5
# Weight    
ipw[1] = 5.0/9.0
ipw[2] = 8.0/9.0
ipw[3] = 5.0/9.0

0.5555555555555556

In [72]:
# Surface nodes shape functions and derivatives
nnfa = 3 # assume quadratic element
N    = zeros(nip,nnfa,1)
dNdx = zeros(nip,nnfa,1)
for i=1:nip
    ξ = ipx[i]
    if nipfa==3
        N[i,:,:]    .= [ξ/2*(ξ-1)  1.0-ξ^2 ξ/2*(ξ+1)]'
        dNdx[i,:,:] .= [ξ-1.0/2.0 -2.0*ξ   ξ+1.0/2.0]'  #w.r.t η2
    end
end

In [73]:
# Traction vector
nipfa = nnfa
T = zeros(3,2)
T[:,2] .= 1.0 # only in y
Fex     = zeros(8)
Fey     = zeros(8)
Fs      = zeros(2)
Ff      = zeros(2*nnfa)
f2n     = [2 5 1]
xn      = [-1.0 0.0 1.0]
yn      = [-1.0 1.0 1.0]
x       = [xn; yn]'
for ip=1:nipfa 
    Ni    = N[ip,:]
    dNdXi = dNdx[ip,:,1] 
    J     = x' * dNdXi
    detJ  = 1.0#sqrt( J[1]^2 + J[2]^2 )
    
    # Surface tractions 
    Fs[1] = Ni'*T[:,1]  # use shape functions to evaluate traction at integration point
    Fs[2] = Ni'*T[:,2]
    # Integrate surface traction contributions
    Ff[1:nnfa]       .+= ipw[ip] .* detJ .* Ni .* Fs[1]
    Ff[(nnfa+1):end] .+= ipw[ip] .* detJ .* Ni .* Fs[2]  
end
Fex[f2n] .= Ff[1:nnfa]'
Fey[f2n] .= Ff[(nnfa+1):end]'
# display([Fex; Fey])
display(Ff)


6-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.3336805555555555
 1.3326388888888885
 0.33368055555555554